# Black Scholes Exercise 3: Numexpr implementation

- Use cProfile and Line Profiler to look for bottlenecks and hotspots in the code

In [1]:
# Boilerplate for the example

import cProfile
import pstats
import numpy as np
%load_ext line_profiler

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
price, strike, t = gen_data(nopt)
call = np.zeros(nopt, dtype=np.float64)
put  = -np.ones(nopt, dtype=np.float64)

# The numexpr Black Scholes algorithm

How does this differ from the NumPy and Naive variants?

In [2]:
import numexpr as ne

def black_scholes (nopt, price, strike, t, rate, vol ):
    mr = -rate
    sig_sig_two = vol * vol * 2

    P = price
    S = strike
    T = t

    a = ne.evaluate("log(P / S) ")
    b = ne.evaluate("T * mr ")

    z = ne.evaluate("T * sig_sig_two ")
    c = ne.evaluate("0.25 * z ")
    y = ne.evaluate("1/sqrt(z) ")

    w1 = ne.evaluate("(a - b + c) * y ")
    w2 = ne.evaluate("(a - b - c) * y ")

    d1 = ne.evaluate("0.5 + 0.5 * erf(w1) ")
    d2 = ne.evaluate("0.5 + 0.5 * erf(w2) ")

    Se = ne.evaluate("exp(b) * S ")

    call = ne.evaluate("P * d1 - Se * d2 ")
    put = ne.evaluate("call - P + Se ")

    return call, put

ne.set_num_threads(ne.detect_number_of_cores())
ne.set_vml_accuracy_mode('high')

'high'

## Run timeit, cProfile, and line_profiler to see what is happening

In [3]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

4.2 ms ± 283 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
cProfile.run('black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)')

         268 function calls in 0.006 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.006    0.006 <ipython-input-2-9208cb827f7e>:3(black_scholes)
        1    0.000    0.000    0.006    0.006 <string>:1(<module>)
       12    0.000    0.000    0.000    0.000 necompiler.py:518(getContext)
       27    0.000    0.000    0.000    0.000 necompiler.py:670(getType)
       12    0.000    0.000    0.000    0.000 necompiler.py:709(getArguments)
       12    0.006    0.000    0.006    0.001 necompiler.py:734(evaluate)
       12    0.000    0.000    0.000    0.000 necompiler.py:790(<listcomp>)
       27    0.000    0.000    0.000    0.000 numeric.py:463(asarray)
        1    0.000    0.000    0.006    0.006 {built-in method builtins.exec}
       12    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
       12    0.000    0.000    0.000    0.000 {built-in method builtins.sorted}
  

In [5]:
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

## Crunching the commands down

What is different in this variant of Black Scholes?

In [6]:
def black_scholes(price, strike, t, rate, vol ):
    mr = -rate
    sig_sig_two = vol * vol * 2
    
    P = price
    S = strike
    T = t

    call = ne.evaluate("P * (0.5 + 0.5 * erf((log(P / S) - T * mr + 0.25 * T * sig_sig_two) * 1/sqrt(T * sig_sig_two))) - S * exp(T * mr) * (0.5 + 0.5 * erf((log(P / S) - T * mr - 0.25 * T * sig_sig_two) * 1/sqrt(T * sig_sig_two))) ")
    put = ne.evaluate("call - P + S * exp(T * mr) ")
    
    return call, put
        

ne.set_num_threads(ne.detect_number_of_cores())
ne.set_vml_accuracy_mode('high')

'high'

## Run timeit, cProfile, and line_profiler to see what is happening

In [7]:
%timeit black_scholes( price, strike, t, RISK_FREE, VOLATILITY)

2.2 ms ± 263 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
cProfile.run('black_scholes(price, strike, t, RISK_FREE, VOLATILITY)')

         70 function calls in 0.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.004    0.004 <ipython-input-6-b8361bbcd286>:1(black_scholes)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 necompiler.py:518(getContext)
       10    0.000    0.000    0.000    0.000 necompiler.py:670(getType)
        2    0.000    0.000    0.000    0.000 necompiler.py:709(getArguments)
        2    0.004    0.002    0.004    0.002 necompiler.py:734(evaluate)
        2    0.000    0.000    0.000    0.000 necompiler.py:790(<listcomp>)
       10    0.000    0.000    0.000    0.000 numeric.py:463(asarray)
        1    0.000    0.000    0.004    0.004 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.sorted}
   

In [9]:
%lprun -f black_scholes black_scholes(price, strike, t, RISK_FREE, VOLATILITY)